In [1]:
%reload_ext autoreload
%reload_ext autoreload

In [2]:
import time
import os
import sys
import copy
import time
import datetime
import random
import math
import warnings
from functools import partial
# warnings.filterwarnings('ignore')

In [3]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from torch import Tensor
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn

In [4]:
import numpy as np
import scipy
import seaborn as sns
import pandas as pd
from tqdm.notebook import tqdm, trange

In [5]:
sys.path.append("../")

In [6]:
from chaosmining.data_utils import read_formulas, create_simulation_data
from chaosmining.simulation.models import MLPRegressor
from chaosmining.simulation.functions import abs_argmax_topk
from chaosmining.utils import radar_factory

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, precision_score, recall_score, accuracy_score, roc_curve, auc, balanced_accuracy_score

In [8]:
from captum.attr import IntegratedGradients, Saliency, DeepLift, FeatureAblation

In [9]:
import matplotlib
# mpl.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt 
from matplotlib.colors import ListedColormap

matplotlib.rcParams['lines.linewidth'] = 1
matplotlib.rcParams['lines.markersize'] = 5
plt.rcParams['figure.figsize'] = [4, 4]

# Santander customer dataset

In [10]:
house_train_path = '../data/house_prices_advanced_regression/train.csv'
house_test_path = '../data/house_prices_advanced_regression/test.csv'

In [11]:
customer_train_path = '../data/santander_customer_satisfaction/train.csv'
customer_test_path = '../data/santander_customer_satisfaction/test.csv'
sample_path = '../data/santander_customer_satisfaction/sample_submission.csv'

In [12]:
train  = pd.read_csv(customer_train_path,index_col=0)
test   = pd.read_csv(customer_test_path, index_col=0)
sample = pd.read_csv(sample_path)

In [13]:
train.dtypes.value_counts()

int64      259
float64    111
Name: count, dtype: int64

In [14]:
train.isnull().values.any()

False

In [15]:
train.select_dtypes(include=['int64']).nunique()

var3                             208
var15                            100
ind_var1_0                         2
ind_var1                           2
ind_var2_0                         1
                                ... 
num_var45_ult3                   172
saldo_var2_ult1                    1
saldo_medio_var13_medio_hace3      1
saldo_medio_var13_medio_ult1       3
TARGET                             2
Length: 259, dtype: int64

a good many of the integer features have one single value. Such columns have zero variance and thus have no predictive value, In https://www.kaggle.com/code/carlmcbrideellis/tabular-classification-with-neural-networks-keras/notebook they drop these columns from the train, as well as the test data to maintain consistency.

In [16]:
# features_to_drop = train.nunique()
# features_to_drop = features_to_drop.loc[features_to_drop.values==1].index
# # now drop these columns from both the training and the test datasets
# train = train.drop(features_to_drop,axis=1)
# test  = test.drop(features_to_drop,axis=1)

In [17]:
train.dtypes.value_counts()

int64      259
float64    111
Name: count, dtype: int64

In [18]:
train.shape

(76020, 370)

In [19]:
X = train.iloc[:,:-1]
y = train['TARGET']

In [20]:
from imblearn.under_sampling import RandomUnderSampler

In [21]:
rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(X, y)
X_resampled = X_resampled.to_numpy()
y_resampled = y_resampled.to_numpy().reshape(y_resampled.shape[0],-1)

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, 
                                                  train_size=0.8,
                                                  test_size=0.2, 
                                                  random_state=42, 
                                                  shuffle=True)

In [23]:
from sklearn.preprocessing import MinMaxScaler
scaler  = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test   = scaler.transform(X_test)
# test    = scaler.transform(test)

# Training and Test a MLP

In [131]:
train_set = TensorDataset(Tensor(X_train), Tensor(y_train))
train_loader = DataLoader(train_set, batch_size=1000, shuffle=True)
test_set = TensorDataset(Tensor(X_test), Tensor(y_test))
test_loader = DataLoader(test_set, batch_size=y_test.shape[0])

In [132]:
hidden_layer_sizes = (100,100,100)
num_epochs = 500

In [24]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
# device = 'cpu'

In [173]:
class LinearResBlock(nn.Module):
    def __init__(self, in_channels: int, out_channels: int):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_channels, out_channels),
            nn.ReLU(),
        )

    def forward(self, x: Tensor)-> Tensor:
        out = self.net(x)
        return out-x

In [25]:
from typing import List

class LinearBlock(nn.Module):
    def __init__(self, in_channels: int, out_channels: int):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_channels, out_channels),
            nn.ReLU(),
        )

    def forward(self, x: Tensor)-> Tensor:
        x = self.net(x)
        return x
    
class MLPClassifier(nn.Module):
    def __init__(self, in_channels: int, sizes: List[int], p: float=0.0):
        super().__init__()
        self.blocks = nn.ModuleList([
            LinearBlock(in_channels, sizes[0]),
            *[LinearBlock(sizes[i], sizes[i+1]) for i in range(len(sizes)-1)]
        ])
        self.dropout = nn.Dropout(p)
        self.project = nn.Sequential(
            nn.Linear(sizes[-1], 1),
            nn.Sigmoid()
        )
    def forward(self, x: Tensor)-> Tensor:
        for block in self.blocks:
            x = block(x)
        x = self.dropout(x)
        x = self.project(x)
        return x

class MLPCReslassifier(nn.Module):
    def __init__(self, in_channels: int, sizes: List[int], p: float=0.0):
        super().__init__()
        self.blocks = nn.ModuleList([
            LinearBlock(in_channels, sizes[0]),
            *[LinearResBlock(sizes[i], sizes[i+1]) for i in range(len(sizes)-1)]
        ])
        self.dropout = nn.Dropout(p)
        self.project = nn.Sequential(
            nn.Linear(sizes[-1], 1),
            nn.Sigmoid()
        )
    def forward(self, x: Tensor)-> Tensor:
        for block in self.blocks:
            x = block(x)
        x = self.dropout(x)
        x = self.project(x)
        return x

In [141]:
p=0
model = MLPClassifier(X_train.shape[-1], hidden_layer_sizes, p=p)
model.to(device)
model.train()

MLPClassifier(
  (blocks): ModuleList(
    (0): LinearBlock(
      (net): Sequential(
        (0): Linear(in_features=369, out_features=100, bias=True)
        (1): ReLU()
      )
    )
    (1-2): 2 x LinearBlock(
      (net): Sequential(
        (0): Linear(in_features=100, out_features=100, bias=True)
        (1): ReLU()
      )
    )
  )
  (dropout): Dropout(p=0, inplace=False)
  (project): Sequential(
    (0): Linear(in_features=100, out_features=1, bias=True)
    (1): Sigmoid()
  )
)

In [142]:
# criterion = nn.MSELoss(reduction='mean')
criterion = nn.BCELoss(reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), 0.001)

In [26]:
def train(model, dataloader, num_epochs, optimizer):
    pbar = trange(num_epochs, desc='Train', unit='epoch', initial=0, disable=False)
    for epoch in pbar:  # loop over the dataset multiple times
        running_loss = 0.0
        for inputs, targets in dataloader:
            # zero the parameter gradients
            optimizer.zero_grad()
            inputs = inputs.to(device)
            targets = targets.to(device)
            # forward pass
            outputs = model(inputs)
            # defining loss
            # print(outputs.dtype,targets.dtype)
            # print(outputs.shape,targets.shape)
            # print(outputs[0],targets[0])
            loss = criterion(outputs, targets)

            # computing gradients
            loss.backward()
            # accumulating running loss
            running_loss += loss.item()
            # updated weights based on computed gradients
            optimizer.step()
        pbar.set_postfix(loss = '%.3f' % running_loss)
        # print(loss.item())
    print('train loss:', running_loss)

In [144]:
train(model, train_loader, num_epochs, optimizer)

Train:   0%|          | 0/500 [00:00<?, ?epoch/s]

train loss: 1.6263459026813507


In [145]:
y_pred = model(Tensor(X_test).to(device)).detach().cpu().numpy()

y_test =y_test.astype(int)
fpr, tpr, thresholds = roc_curve(y_test,y_pred)
ACC = balanced_accuracy_score(y_test,np.around(y_pred))
print(f'With Drop out{p}')
print('Test auc is', auc(fpr, tpr))
print('Balanced_accuracy_score is', ACC)

y_pred_train = model(Tensor(X_train).to(device)).detach().cpu().numpy()


fpr, tpr, thresholds = roc_curve(y_train,y_pred_train)
ACC = balanced_accuracy_score(y_train,np.around(y_pred_train))
print(f'With Drop out:{p}')
print('Train auc is', auc(fpr, tpr))
print('Balanced_accuracy_score of Train is', ACC)

With Drop out0
Test auc is 0.7497733729838487
Balanced_accuracy_score is 0.722542202372398
With Drop out:0
Train auc is 0.9339094995732742
Balanced_accuracy_score of Train is 0.8581407884234641


# play with dropout

In [146]:
for p in [0,0.1,0.2,0.3,0.4,0.5]:
    train_set = TensorDataset(Tensor(X_train), Tensor(y_train))
    train_loader = DataLoader(train_set, batch_size=1000, shuffle=True)
    test_set = TensorDataset(Tensor(X_test), Tensor(y_test))
    test_loader = DataLoader(test_set, batch_size=y_test.shape[0],shuffle=False)
    hidden_layer_sizes = (100,100,100)
    num_epochs = 500
    model = MLPClassifier(X_train.shape[-1], hidden_layer_sizes, p=p)
    model.to(device)
    model.train()
    # print(model)
    criterion = nn.BCELoss(reduction='mean')
    optimizer = torch.optim.Adam(model.parameters(), 0.001)
    train(model, train_loader, num_epochs, optimizer)
    model.eval()
    y_pred = model(Tensor(X_test).to(device)).detach().cpu().numpy()
    
    y_test =y_test.astype(int)
    
    fpr, tpr, thresholds = roc_curve(y_test,y_pred)
    ACC = balanced_accuracy_score(y_test,np.around(y_pred))
    print(f'With Drop out{p}')
    print('Test auc is', auc(fpr, tpr))
    print('Balanced_accuracy_score is', ACC)

    y_pred_train = model(Tensor(X_train).to(device)).detach().cpu().numpy()
    
    
    fpr, tpr, thresholds = roc_curve(y_train,y_pred_train)
    ACC = balanced_accuracy_score(y_train,np.around(y_pred_train))
    print(f'With Drop out:{p}')
    print('Train auc is', auc(fpr, tpr))
    print('Balanced_accuracy_score of Train is', ACC)


Train:   0%|          | 0/500 [00:00<?, ?epoch/s]

train loss: 1.7384424209594727
With Drop out0
Test auc is 0.7531921244348142
Balanced_accuracy_score is 0.7188830051847839
With Drop out:0
Train auc is 0.9312184490675954
Balanced_accuracy_score of Train is 0.8543059572306704


Train:   0%|          | 0/500 [00:00<?, ?epoch/s]

train loss: 1.605219841003418
With Drop out0.1
Test auc is 0.746643432791271
Balanced_accuracy_score is 0.7244685320096842
With Drop out:0.1
Train auc is 0.936305556419373
Balanced_accuracy_score of Train is 0.8558965906852837


Train:   0%|          | 0/500 [00:00<?, ?epoch/s]

...

train loss: 1.6422038078308105
With Drop out0.3
Test auc is 0.7587541594348695
Balanced_accuracy_score is 0.7275362879600252
With Drop out:0.3
Train auc is 0.9337864919682252
Balanced_accuracy_score of Train is 0.8537384291652413


Train:   0%|          | 0/500 [00:00<?, ?epoch/s]

train loss: 1.6573023796081543
With Drop out0.4
Test auc is 0.7607482008025914
Balanced_accuracy_score is 0.7190847585040406
With Drop out:0.4
Train auc is 0.9358712290049168
Balanced_accuracy_score of Train is 0.858726456655368


Train:   0%|          | 0/500 [00:00<?, ?epoch/s]

train loss: 1.6352637112140656
With Drop out0.5
Test auc is 0.7600240998485469
Balanced_accuracy_score is 0.7266049061985252
With Drop out:0.5
Train auc is 0.9351433764205478
Balanced_accuracy_score of Train is 0.8580760021146252


In [169]:
y_train.shape

(4812, 1)

# MLP res Block

In [182]:
for p in [0,0.1,0.2,0.3,0.4,0.5]:
    for hidden_layer_sizes in [(100,100,100),(100,100,100,100),(100,100,100,100,100),(100,100,100,100,100,100)]:
        train_set = TensorDataset(Tensor(X_train), Tensor(y_train))
        train_loader = DataLoader(train_set, batch_size=1000, shuffle=True)
        test_set = TensorDataset(Tensor(X_test), Tensor(y_test))
        test_loader = DataLoader(test_set, batch_size=y_test.shape[0],shuffle=False)
        num_epochs = 500
        # model = MLPClassifier(X_train.shape[-1], hidden_layer_sizes, p=p)
        model = MLPCReslassifier(X_train.shape[-1], hidden_layer_sizes)
        model.to(device)
        model.train()
        
        criterion = nn.BCELoss(reduction='mean')
        optimizer = torch.optim.Adam(model.parameters(), 0.001)
        train(model, train_loader, num_epochs, optimizer)
        model.eval()
        y_pred = model(Tensor(X_test).to(device)).detach().cpu().numpy()
        
        y_test = y_test.astype(int)
        
        fpr, tpr, thresholds = roc_curve(y_test,y_pred)
        ACC = balanced_accuracy_score(y_test,np.around(y_pred))
        print(f'With Drop out{p}')
        print('Test auc is', auc(fpr, tpr))
        print('Balanced_accuracy_score is', ACC)
        
        y_pred_train = model(Tensor(X_train).to(device)).detach().cpu().numpy()
        fpr, tpr, thresholds = roc_curve(y_train,y_pred_train)
        ACC = balanced_accuracy_score(y_train,np.around(y_pred_train))
        print(f'With Drop out:{p}')
        print(f'hidden_layer_sizes:{hidden_layer_sizes}')
        print('Train auc is', auc(fpr, tpr))
        print('Balanced_accuracy_score of Train is', ACC)

Train:   0%|          | 0/500 [00:00<?, ?epoch/s]

train loss: 1.7860514223575592
With Drop out0
Test auc is 0.7493132095912975
Balanced_accuracy_score is 0.7169539118033983
With Drop out:0
hidden_layer_sizes:(100, 100, 100)
Train auc is 0.9318143103454233
Balanced_accuracy_score of Train is 0.8528946522789231


Train:   0%|          | 0/500 [00:00<?, ?epoch/s]

train loss: 1.5011220872402191
With Drop out0
Test auc is 0.746303492267044
Balanced_accuracy_score is 0.724134118973656
With Drop out:0
hidden_layer_sizes:(100, 100, 100, 100)
Train auc is 0.9433249370277077
Balanced_accuracy_score of Train is 0.8617600107804417


Train:   0%|          | 0/500 [00:00<?, ?epoch/s]

...

train loss: 1.524756371974945
With Drop out0.5
Test auc is 0.7503413223962767
Balanced_accuracy_score is 0.7220751296195983
With Drop out:0.5
hidden_layer_sizes:(100, 100, 100, 100)
Train auc is 0.9409288801816091
Balanced_accuracy_score of Train is 0.8638399104394066


Train:   0%|          | 0/500 [00:00<?, ?epoch/s]

train loss: 1.4353521764278412
With Drop out0.5
Test auc is 0.7376377726433554
Balanced_accuracy_score is 0.726201399560012
With Drop out:0.5
hidden_layer_sizes:(100, 100, 100, 100, 100)
Train auc is 0.9477549384443685
Balanced_accuracy_score of Train is 0.8724253920867411


Train:   0%|          | 0/500 [00:00<?, ?epoch/s]

train loss: 1.489148736000061
With Drop out0.5
Test auc is 0.7483956465502946
Balanced_accuracy_score is 0.7210829454879114
With Drop out:0.5
hidden_layer_sizes:(100, 100, 100, 100, 100, 100)
Train auc is 0.9477419811826007
Balanced_accuracy_score of Train is 0.8745923645447855


# Iterative Method

In [75]:
y_test

array([[0],
       [0],
       [1],
       ...,
       [1],
       [1],
       [0]])

In [30]:
import pickle

In [31]:
for repeat in range(1,6):
    for xai_method in [Saliency,DeepLift,FeatureAblation,IntegratedGradients]:
        print(xai_method.get_name())
        num_epochs = 500
        hidden_layer_sizes = (100,100,100)
        reduce_rate = 0.5
        best_score = 0
        num_cur_features = X_train.shape[-1]
        select_arr = np.ones(num_cur_features)
        remaining_inds = np.nonzero(select_arr)[0]
        num_select = 3
        num_list = []
        auc_list = []
        acc_list = []
        feature_list = []
        resuming = True
        start_time = time.time()
        while resuming:
            num_list.append(num_cur_features)
            bool_arr = np.array(select_arr, dtype='bool') 
            # print(X_train[...,bool_arr].shape)
            # torch.manual_seed(42)
            train_set = TensorDataset(Tensor(X_train[...,bool_arr]), Tensor(y_train))
            train_loader = DataLoader(train_set, batch_size=y_train.shape[0], shuffle=True)
            test_set = TensorDataset(Tensor(X_test[...,bool_arr]), Tensor(y_test))
            test_loader = DataLoader(test_set, batch_size=y_test.shape[0])
        
            model = MLPClassifier(int(np.sum(select_arr)), hidden_layer_sizes, p=0.0)
            model.to(device)
            model.train()
        
            # criterion = nn.MSELoss(reduction='mean')
            criterion = nn.BCELoss(reduction='mean')
            optimizer = torch.optim.Adam(model.parameters(), 0.001)

            train(model, train_loader, num_epochs, optimizer)
        
            model.eval()
            y_pred = model(Tensor(X_test[...,bool_arr]).to(device)).detach().cpu().numpy()
            fpr, tpr, thresholds = roc_curve(y_test,y_pred)
            auc_score = auc(fpr, tpr)
            # print('Test auc is', auc_score)
            acc = balanced_accuracy_score(y_test,np.around(y_pred))
            # print('Test ACC is', acc)
            print(f'With {num_cur_features},{xai_method.get_name()} get ACC:{acc} AUC:{auc_score}')
            acc_list.append(acc)
            auc_list.append(auc_score)
            feature_list.append(np.where(select_arr==1)[0])
            
            xai = xai_method(model)
        
            num_remove = int(num_cur_features*(1-reduce_rate))
            if num_cur_features - num_remove<=num_select:
                num_remove = num_cur_features - num_select
            # print(num_remove)
            # print('num_remove', num_remove)
            xai_attr_test = xai.attribute(Tensor(X_test[...,bool_arr]).to(device))
            abs_xai_attr_test = np.abs(xai_attr_test.detach().cpu().numpy()).mean(0)
            inds = np.argpartition(abs_xai_attr_test, num_remove)[:num_remove]
            inds_to_remove = remaining_inds[inds]
            select_arr[inds_to_remove] = 0
            
            remaining_inds = np.nonzero(select_arr)[0]
            num_cur_features -= num_remove
            if num_remove == 0:
                resuming = False
        # np.save(f'./result/{xai_method.get_name()}_feature_final.npy',np.where(select_arr==1)[0])
        elapsed_time = time.time() - start_time
        dict_method = {}
        dict_method['auc'] = auc_list
        dict_method['acc'] = acc_list
        dict_method['feat'] = feature_list
        dict_method['time'] = elapsed_time
        with open(f'./result/{xai_method.get_name()}_{repeat}.pkl', 'wb') as f:
            pickle.dump(dict_method, f)

Saliency


Train:   0%|          | 0/500 [00:00<?, ?epoch/s]

train loss: 0.38475772738456726
With 369,Saliency get ACC:0.7141597665188986 AUC:0.7606487060150127


/home/zwkan/ChaosMining/src/captum/captum/_utils/gradient.py:57: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  warnings.warn(


Train:   0%|          | 0/500 [00:00<?, ?epoch/s]

...

train loss: 0.496622234582901
With 12,Integrated Gradients get ACC:0.7074991432393292 AUC:0.7895450324463558


Train:   0%|          | 0/500 [00:00<?, ?epoch/s]

train loss: 0.5303881764411926
With 6,Integrated Gradients get ACC:0.6946007495273998 AUC:0.7747120178648419


Train:   0%|          | 0/500 [00:00<?, ?epoch/s]

train loss: 0.5376461744308472
With 3,Integrated Gradients get ACC:0.6926633649137159 AUC:0.7713029395182242


In [46]:
feature_saliency = np.load('Saliency_feature.npy')

In [47]:
feature_saliency.shape

(73,)

In [48]:
bool_arr = np.array(select_arr, dtype='bool')

In [49]:
X_test[...,bool_arr].shape

(1204, 73)

In [50]:
select_arr.shape

(369,)

In [51]:
print('The best score is:', best_score)
print('best features:', np.where(select_arr==1)[0])

The best score is: 0
best features: [  0   1  14  17  23  24  27  28  29  38  50  51  52  54  55  62  63  73
  74  75  76  79  88  89  90  93  94 125 127 129 130 137 138 147 151 154
 157 158 164 166 193 254 255 256 257 258 259 273 274 275 276 277 278 279
 280 281 289 291 297 298 299 300 301 302 311 323 324 325 326 330 332 334
 368]


In [52]:
from sklearn.ensemble import GradientBoostingClassifier

# XGboost and SVM

In [ ]:
clf = GradientBoostingClassifier(n_estimators=500, learning_rate=0.05, random_state=156).fit(X_train, np.squeeze(y_train))
clf.score(X_test, np.squeeze(y_test))

In [ ]:
y_pred_proba = clf.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test,y_pred_proba)
print('Test auc is', auc(fpr, tpr))
acc = balanced_accuracy_score(y_test,np.around(y_pred_proba))
print(f'balanced accuarcy score:{acc}')

In [ ]:
from sklearn.svm import SVC

In [ ]:
clf = SVC(gamma='auto')

In [ ]:
clf.fit(X_train, np.squeeze(y_train))
y_pred_proba = clf.predict(X_test)
fpr, tpr, thresholds = roc_curve(y_test,y_pred_proba)
print('Test auc is', auc(fpr, tpr))
acc = balanced_accuracy_score(y_test,np.around(y_pred_proba))
print(f'balanced accuarcy score:{acc}')

# logistic Regression

In [71]:
clf = LogisticRegression(penalty=None)
clf.fit(X_train, np.squeeze(y_train))
y_pred_proba = clf.predict_proba(X_test)[:, 1]
print(y_pred_proba)
fpr, tpr, thresholds = roc_curve(y_test,y_pred_proba)
print('Test auc is', auc(fpr, tpr))
acc = balanced_accuracy_score(y_test,np.around(y_pred_proba))
print(f'balanced accuarcy score:{acc}')

[0.58345724 0.26040545 0.67795407 ... 0.84154498 0.68576292 0.18908735]
Test auc is 0.7917090440761909
balanced accuarcy score:0.7216080568667986


/home/zwkan/miniconda3/envs/CM/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# L1 penalty

In [55]:
from sklearn.linear_model import LogisticRegression

In [56]:
clf = LogisticRegression(penalty='l1',solver='liblinear')
clf.fit(X_train, np.squeeze(y_train))
y_pred_proba = clf.predict_proba(X_test)[:, 1]
print(y_pred_proba)
fpr, tpr, thresholds = roc_curve(y_test,y_pred_proba)
print('Test auc is', auc(fpr, tpr))
acc = balanced_accuracy_score(y_test,np.around(y_pred_proba))
print(f'balanced accuarcy score:{acc}')

[0.57961152 0.2762406  0.67486701 ... 0.82669343 0.65260206 0.20646377]
Test auc is 0.7894441557867273
balanced accuarcy score:0.7065677614778292


In [57]:
coef = clf.coef_

array([0.])

In [60]:
np.where(coef!=0)[1]

array([  0,   1,   2,  12,  13,  17,  23,  24,  27,  28,  29,  30,  32,
        38,  45,  46,  48,  49,  50,  54,  55,  62,  63,  64,  73,  74,
        75,  76,  78,  81,  87,  89,  93,  94, 111, 113, 114, 147, 149,
       150, 153, 156, 158, 164, 166, 182, 193, 196, 208, 224, 255, 259,
       263, 273, 274, 276, 277, 280, 281, 284, 289, 290, 291, 309, 310,
       312, 324, 326, 328, 330, 332, 368])

In [61]:
non_zero_coef = coef[:,np.where(coef!=0)[1]]

In [62]:
non_zero_coef.shape

(1, 72)

# RFE logstic regression

In [183]:
from sklearn.feature_selection import RFE

In [ ]:
num_list = []
auc_list = []
acc_list = []
feature_list = []
for num_feature in [369,185,93,47,24,12,6,3]:
    estimator = LogisticRegression(penalty=None)
    selector = RFE(estimator, n_features_to_select=5, step=1)
    selector = selector.fit(X_train, np.squeeze(y_train))
    selected_RFE = selector.ranking_
    selected_RFE[selected_RFE != 1] = 0
    bool_arr = np.array(selected_RFE, dtype='bool') 
    X_train_selected = X_train[...,bool_arr]
    X_test_selected = X_test[...,bool_arr]
    clf = LogisticRegression(penalty=None)
    clf.fit(X_train_selected, np.squeeze(y_train))
    
    y_pred_proba_linear = clf.predict_proba(X_test_selected)[:,1]
    fpr, tpr, thresholds = roc_curve(y_test,y_pred_proba_linear)
    
    acc = balanced_accuracy_score(y_test,np.around(y_pred_proba_linear))
    print(f'With {num_feature} in RFE')
    auc_score = auc(fpr, tpr)
    print('Test auc is', auc_score)
    print(f'balanced accuarcy score:{acc}')
    num_list.append(num_feature)
    acc_list.append(acc)
    auc_list.append(auc_list)

dict_method = {}
dict_method['auc'] = auc_list
dict_method['acc'] = acc_list
dict_method['feat'] = feature_list
with open(f'./result/linear.pkl', 'wb') as f:
    pickle.dump(dict_method, f)

# RFE svc

In [192]:
from sklearn.svm import SVC

In [196]:
estimator = SVC(                    
            kernel = 'linear',
            probability = True,
            random_state = 42) 
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, np.squeeze(y_train))

selected_RFE = selector.ranking_
selected_RFE[selected_RFE != 1] = 0

bool_arr = np.array(selected_RFE, dtype='bool') 
X_train_selected = X_train[...,bool_arr]
X_test_selected = X_test[...,bool_arr]

X_train_selected.shape

clf = LogisticRegression(penalty=None)
clf.fit(X_train_selected, np.squeeze(y_train))

y_pred_proba_linear = clf.predict_proba(X_test_selected)[:,1]
fpr, tpr, thresholds = roc_curve(y_test,y_pred_proba_linear)
print('Test auc is', auc(fpr, tpr))
acc = balanced_accuracy_score(y_test,np.around(y_pred_proba_linear))
print(f'balanced accuarcy score:{acc}')

Test auc is 0.48835772798125077
balanced accuarcy score:0.4937235371502482


# Decision Tree

In [195]:
estimator = DecisionTreeClassifier()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, np.squeeze(y_train))

selected_RFE = selector.ranking_
selected_RFE[selected_RFE != 1] = 0

bool_arr = np.array(selected_RFE, dtype='bool') 
X_train_selected = X_train[...,bool_arr]
X_test_selected = X_test[...,bool_arr]

X_train_selected.shape

clf = LogisticRegression(penalty=None)
clf.fit(X_train_selected, np.squeeze(y_train))

y_pred_proba_linear = clf.predict_proba(X_test_selected)[:,1]
fpr, tpr, thresholds = roc_curve(y_test,y_pred_proba_linear)
print('Test auc is', auc(fpr, tpr))
acc = balanced_accuracy_score(y_test,np.around(y_pred_proba_linear))
print(f'balanced accuarcy score:{acc}')

Test auc is 0.7285395270681098
balanced accuarcy score:0.664920901643875


array([ 13,  16, 276])

# Curve with repetition

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.feature_selection import RFE

In [73]:
# def get_estimator(method):
#     if method =='linear':
#         estimator = LogisticRegression(penalty=None)
#     elif method == 'SVM':
#         estimator = SVC(kernel = 'linear',probability = True) 
#     elif method =='DT':
#         estimator = DecisionTreeClassifier()
#     else:
#         raise 'wrong method'
#     return estimator
# for repeat in range(2,6):
#     # for method in ['linear','DT']:
#     for method in ['DT']:
#         auc_list = []
#         acc_list = []
#         feature_list = []
#         for num_feature in [369,185,93,47,24,12,6,3]:
#             estimator = get_estimator(method)
#             selector = RFE(estimator, n_features_to_select=num_feature)
#             selector = selector.fit(X_train, np.squeeze(y_train))
#             selected_RFE = selector.ranking_
#             selected_RFE[selected_RFE != 1] = 0
#             bool_arr = np.array(selected_RFE, dtype='bool') 
#             X_train_selected = X_train[...,bool_arr]
#             X_test_selected = X_test[...,bool_arr]
#             clf = get_estimator(method)
#             clf.fit(X_train_selected, np.squeeze(y_train))
            
#             y_pred_proba_linear = clf.predict_proba(X_test_selected)[:,1]
#             fpr, tpr, thresholds = roc_curve(y_test,y_pred_proba_linear)
            
#             acc = balanced_accuracy_score(y_test,np.around(y_pred_proba_linear))
#             print(f'With {num_feature} in RFE')
#             auc_score = auc(fpr, tpr)
#             print('Test auc is', auc_score)
#             print(f'balanced accuarcy score:{acc}')

#             acc_list.append(acc)
#             auc_list.append(auc_score)
#             feature_list.append(np.where(selected_RFE==1)[0])
#             # print(np.where(selected_RFE==1)[0])
#             print(f'With{num_feature},{method} get ACC:{acc} AUC:{auc_score}')
#         dict_method = {}
#         dict_method['auc'] = auc_list
#         dict_method['acc'] = acc_list
#         dict_method['feat'] = feature_list
#         print(dict_method)
#         # with open(f'./result/{method}_{repeat}.pkl', 'wb') as f:
#         #     pickle.dump(dict_method, f)

In [74]:
def get_result(name):
    acc_curve = []
    auc_curve = []
    feature_curve = []
    for i in range(1,6):
        with open(f'./result/{name}_{i}.pkl','rb') as file:
            i_th_result = pickle.load(file)
            acc_curve.append(i_th_result['acc'])
            auc_curve.append(i_th_result['auc'])
            feature_curve.append(i_th_result['feat'])
    return acc_curve, auc_curve, feature_curve

In [64]:
import time

def get_estimator(method):
    if method =='linear':
        estimator = LogisticRegression(penalty=None)
    elif method == 'SVM':
        estimator = SVC(kernel = 'linear',probability = True) 
    elif method =='DT':
        estimator = DecisionTreeClassifier()
    else:
        raise 'wrong method'
    return estimator
    
for method in ['DT','SVM','linear']:
    for repeat in range(1,6):
        reduce_rate = 0.5
        num_cur_features = X_train.shape[-1]
        select_arr = np.ones(num_cur_features)
        remaining_inds = np.nonzero(select_arr)[0]
        num_select = 3
        num_list = []
        auc_list = []
        acc_list = []
        feature_list = []
        resuming = True
        start_time = time.time()
        while resuming:
            bool_arr = np.array(select_arr, dtype='bool') 
            X_train_selected = X_train[...,bool_arr]
            X_test_selected = X_test[...,bool_arr]
            clf = get_estimator(method)
            clf.fit(X_train_selected, np.squeeze(y_train))
            
            y_pred_proba = clf.predict_proba(X_test_selected)[:,1]
            acc = balanced_accuracy_score(y_test,np.around(y_pred_proba))
            fpr, tpr, thresholds = roc_curve(y_test,y_pred_proba)
            

            auc_score = auc(fpr, tpr)
            acc = balanced_accuracy_score(y_test,np.around(y_pred_proba))
            print(f'With {num_cur_features},{method} get ACC:{acc} AUC:{auc_score}')
            acc_list.append(acc)
            auc_list.append(auc_score)
            feature_list.append(np.where(select_arr==1)[0])
            
            num_remove = int(num_cur_features*(1-reduce_rate))
            if num_cur_features - num_remove<=num_select:
                num_remove = num_cur_features - num_select
            num_cur_features -= num_remove
            # print(num_cur_features)
            estimator = get_estimator(method)
            if num_remove == 0:
                resuming = False
                continue
            selector = RFE(estimator, n_features_to_select=num_cur_features,step=num_remove)
            selector = selector.fit(X_train_selected, np.squeeze(y_train))
            selected_RFE = selector.ranking_
            selected_RFE[selected_RFE != 1] = 0
            bool_arr = np.array(selected_RFE, dtype='bool') 

            inds = np.where(selected_RFE==0)[0]
            # print(inds)
            inds_to_remove = remaining_inds[inds]
            # print(inds_to_remove)
            select_arr[inds_to_remove] = 0
            remaining_inds = np.nonzero(select_arr)[0]
            
            # print(remaining_inds,num_cur_features)

        elapsed_time = time.time() - start_time
        dict_method = {}
        dict_method['auc'] = auc_list
        dict_method['acc'] = acc_list
        dict_method['feat'] = feature_list
        dict_method['time'] = elapsed_time
        with open(f'./result/{method}_{repeat}.pkl', 'wb') as f:
            pickle.dump(dict_method, f)

With 369,DT get ACC:0.6675768597234045 AUC:0.6717791326265518
With 185,DT get ACC:0.6566628342748488 AUC:0.6592911549133842
With 93,DT get ACC:0.6724355218501608 AUC:0.6768519849210122
With 47,DT get ACC:0.6822191759620593 AUC:0.6865126524204871
With 24,DT get ACC:0.681420453917331 AUC:0.6858134251633372
With 12,DT get ACC:0.6812822667123606 AUC:0.6822260853223079
With 6,DT get ACC:0.675428656709818 AUC:0.6773162939297124
With 3,DT get ACC:0.648542954110793 AUC:0.6428524050101153
With 369,DT get ACC:0.6666454779619045 AUC:0.6702715102203257
With 185,DT get ACC:0.6589263406922626 AUC:0.6628522391854693
With 93,DT get ACC:0.6627209613407475 AUC:0.6668624318737081
With 47,DT get ACC:0.6816194434924882 AUC:0.6852427120068099
With 24,DT get ACC:0.6721674386725185 AUC:0.6760919552936755
With 12,DT get ACC:0.6578567717257924 AUC:0.6581925666338704
With 6,DT get ACC:0.6708380777607039 AUC:0.672612401472523
With 3,DT get ACC:0.6468791801629503 AUC:0.6410283339045071
With 369,DT get ACC:0.670904

/home/zwkan/miniconda3/envs/CM/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/zwkan/miniconda3/envs/CM/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.

With 185,linear get ACC:0.7136871662779 AUC:0.7916371867296064
With 93,linear get ACC:0.7084968548592149 AUC:0.7908965033109654


/home/zwkan/miniconda3/envs/CM/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/zwkan/miniconda3/envs/CM/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.

With 47,linear get ACC:0.7082315354256719 AUC:0.7868490000773849
With 24,linear get ACC:0.6686491924339741 AUC:0.7397727649601469
With 12,linear get ACC:0.5834816542666681 AUC:0.6410559713455013
With 6,linear get ACC:0.5762931558641122 AUC:0.6382811722696973
With 3,linear get ACC:0.5557972296229148 AUC:0.6026426921078525
With 369,linear get ACC:0.7216080568667986 AUC:0.7917104259482406


...

With 47,linear get ACC:0.7082315354256719 AUC:0.7868490000773849
With 24,linear get ACC:0.6686491924339741 AUC:0.7397727649601469
With 12,linear get ACC:0.5834816542666681 AUC:0.6410559713455013
With 6,linear get ACC:0.5762931558641122 AUC:0.6382811722696973
With 3,linear get ACC:0.5557972296229148 AUC:0.6026426921078525
With 369,linear get ACC:0.7216080568667986 AUC:0.7917104259482406


/home/zwkan/miniconda3/envs/CM/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/zwkan/miniconda3/envs/CM/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.

With 185,linear get ACC:0.7136871662779 AUC:0.7916371867296064
With 93,linear get ACC:0.7084968548592149 AUC:0.7908965033109654


/home/zwkan/miniconda3/envs/CM/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/zwkan/miniconda3/envs/CM/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.

With 47,linear get ACC:0.7082315354256719 AUC:0.7868490000773849
With 24,linear get ACC:0.6686491924339741 AUC:0.7397727649601469
With 12,linear get ACC:0.5834816542666681 AUC:0.6410559713455013
With 6,linear get ACC:0.5762931558641122 AUC:0.6382811722696973
With 3,linear get ACC:0.5557972296229148 AUC:0.6026426921078525


In [89]:
name = 'linear'
Linear_acc, Linear_auc, Linear_feat = get_result(name)

In [90]:
name = 'DT'
DT_acc, DT_auc, DT_feat = get_result(name)

In [91]:
name = 'SVM'
SVM_acc, SVM_auc, SVM_feat = get_result(name)

In [92]:
for i in range(len(SVM_feat[0])):
    print(SVM_feat[0][i].shape)

(369,)
(185,)
(93,)
(47,)
(24,)
(12,)
(6,)
(3,)


In [108]:
# Linear_feat = [284,328,368]
# DT_feat = [1,182,368]
# SVM_feat = [13,16,276]
# feats_RFE = [Linear_feat, DT_feat, SVM_feat]
# name = ['linear','DT','SVM']
feats_RFE = [ DT_feat]
name = ['DT']

In [109]:
for j in range(len(feats_RFE)):

    repeat_times = 6 if name[j]=='DT' else 2
    print(repeat_times)
    for repeat in range(1,repeat_times):
        aucs = []
        accs = []
        method_feat = feats_RFE[j]
        label = name[j]
        for step in range(len(method_feat[0])):
            feat = method_feat[repeat-1][step]
            X_train_select = X_train[...,feat]
            X_test_select = X_test[...,feat]
            print(X_train_select.shape)
            train_set = TensorDataset(Tensor(X_train_select), Tensor(y_train))
            train_loader = DataLoader(train_set, batch_size=1000, shuffle=True)
            test_set = TensorDataset(Tensor(X_test_select), Tensor(y_test))
            test_loader = DataLoader(test_set, batch_size=y_test.shape[0],shuffle=False)
            hidden_layer_sizes = (100,100,100)
            num_epochs = 500
            model = MLPClassifier(X_train_select.shape[-1], hidden_layer_sizes, p=0.0)
            model.to(device)
            model.train()
            # print(model)
            criterion = nn.BCELoss(reduction='mean')
            optimizer = torch.optim.Adam(model.parameters(), 0.001)
            train(model, train_loader, num_epochs, optimizer)
            model.eval()
            y_pred = model(Tensor(X_test_select).to(device)).detach().cpu().numpy()
            
            y_test =y_test.astype(int)
            
            fpr, tpr, thresholds = roc_curve(y_test,y_pred)
            ACC = balanced_accuracy_score(y_test,np.around(y_pred))
            auc_score = auc(fpr, tpr)
            # print(f'With Drop out{p}')
            # print('Test auc is', auc(fpr, tpr))
            # print('Balanced_accuracy_score is', ACC)
            print(f'With feat {feat} from {label}, test auc is:{auc_score} test acc is:{ACC}')
            aucs.append(auc_score)
            accs.append(ACC)
        dict_method = {}
        dict_method['auc'] = aucs
        dict_method['acc'] = accs
        with open(f'./result/feature_{label}_results_{repeat}.pkl', 'wb') as f:
            pickle.dump(dict_method, f)
    # y_pred_train = model(Tensor(X_train).to(device)).detach().cpu().numpy()
    
    
    # fpr, tpr, thresholds = roc_curve(y_train,y_pred_train)
    # ACC = balanced_accuracy_score(y_train,np.around(y_pred_train))
    # print(f'With Drop out:{p}')
    # print('Train auc is', auc(fpr, tpr))
    # print('Balanced_accuracy_score of Train is', ACC)

6
(4812, 369)


Train:   0%|          | 0/500 [00:00<?, ?epoch/s]

train loss: 1.7008539736270905
With feat [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 2

Train:   0%|          | 0/500 [00:00<?, ?epoch/s]

train loss: 1.5745718479156494
With feat [  0   1   2   4   7   8   9  10  11  12  13  14  15  16  17  18  19  20
  21  22  23  27  39  48  49  50  51  52  53  54  55  56  57  58  59  60
  61  62  63  73  74  76  79  81  82  83  84  85  86  87  88  89  90  91
  92  93  94  96 111 112 113 114 115 116 117 118 119 120 121 122 123 124
 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142
 143 144 145 146 147 149 150 151 154 157 159 160 161 162 163 164 166 169
 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 187 190
 193 240 241 254 255 256 257 258 259 273 274 275 276 277 278 279 280 283
 289 291 292 297 298 299 300 301 302 303 304 310 311 320 321 322 323 324
 325 326 327 328 329 330 331 332 334 335 336 337 340 343 346 347 348 349
 350 351 355 356 368] from DT, test auc is:0.7442997777949745 test acc is:0.7230092751251976
(4812, 93)


...

train loss: 2.4481990337371826
With feat [  1 164 182 190 277 289 324 326 328 329 331 368] from DT, test auc is:0.7796425373381828 test acc is:0.7158926340692262
(4812, 6)


Train:   0%|          | 0/500 [00:00<?, ?epoch/s]

train loss: 2.573814183473587
With feat [  1 182 326 329 331 368] from DT, test auc is:0.7751224338636037 test acc is:0.7214726334059276
(4812, 3)


Train:   0%|          | 0/500 [00:00<?, ?epoch/s]

train loss: 2.7070230841636658
With feat [  1 182 368] from DT, test auc is:0.7602451993764994 test acc is:0.7203367345810716


In [104]:
dict_method

{'auc': [0.7531175033441303,
  0.7537352001503476,
  0.7816918535878925,
  0.7933852548724809,
  0.7974092662812164,
  0.7252119791724245,
  0.7132808958952874,
  0.5305545728909868],
 'acc': [0.7119681174480692,
  0.7176199741313553,
  0.7146959328741833,
  0.7292055893960666,
  0.721420122268039,
  0.6973202737212156,
  0.691997302585759,
  0.5245033551853368]}

In [105]:
label

'DT'